<a href="https://colab.research.google.com/github/MatRitchie/Bachelor/blob/main/seq2seq_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installations

In [ ]:
!pip install folium==0.2.1

     |████████████████████████████████| 71kB 3.6MB/s 
  Created wheel for folium: filename=folium-0.2.1-cp36-none-any.whl size=79980 sha256=f1bfd00cd2fbaeb69b83307de50b9ad00bf9c739e22f5cfa2fcd7201ad4c6ed0
  Stored in directory: /root/.cache/pip/wheels/b8/09/f0/52d2ef419c2aaf4fb149f92a33e0008bdce7ae816f0dd8f0c5
Successfully built folium
  Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3


In [ ]:
!pip install --upgrade urllib3

     |████████████████████████████████| 143kB 6.5MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install registrable

In [ ]:
!pip install allennlp

     |████████████████████████████████| 512kB 6.4MB/s 
     |████████████████████████████████| 1.4MB 13.8MB/s 
     |████████████████████████████████| 266kB 28.5MB/s 
     |████████████████████████████████| 317kB 28.0MB/s 
     |████████████████████████████████| 1.1MB 31.9MB/s 
     |████████████████████████████████| 133kB 46.0MB/s 
     |████████████████████████████████| 890kB 43.7MB/s 
     |████████████████████████████████| 2.9MB 40.5MB/s 
     |████████████████████████████████| 133kB 38.9MB/s 
     |████████████████████████████████| 71kB 5.8MB/s 
     |████████████████████████████████| 7.2MB 33.0MB/s 
  Created wheel for jsonnet: filename=jsonnet-0.17.0-cp36-cp36m-linux_x86_64.whl size=3387954 sha256=d8eab9ef0dc80add168637b4034127b34c699dbff5275264320dfc3c52249c86
  Stored in directory: /root/.cache/pip/wheels/26/7a/37/7dbcc30a6b4efd17b91ad1f0128b7bbf84813bd4e1cfb8c1e3
  Created wheel for overrides: filename=overrides-3.1.0-cp36-none-any.whl size=10175 sha256=d2e0a96fd1b5a590ea88db

In [ ]:
!pip install --pre allennlp-models

     |████████████████████████████████| 378kB 5.2MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 71kB 6.0MB/s 
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45613 sha256=095a810befafadb66cb79220599a96210a57bea4729aeb46eaae3fc94c99344c
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b
  Created wheel for word2number: filename=word2number-1.1-cp36-none-any.whl size=5588 sha256=a1331cfcb190381aafbe13df0f3d09e51a3a81bd1ac4ab4abb3df64c38e99ffa
  Stored in directory: /root/.cache/pip/wheels/46/2f/53/5f5c1d275492f2fce1cdab9a9bb12d49286dead829a4078e0e
Successfully built ftfy word2number


# Mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Train model 

In [ ]:
%cd /content/gdrive/My Drive/Bachelor_Project/

/content/gdrive/My Drive/Bachelor_Project


In [ ]:
!allennlp train '/content/gdrive/My Drive/Bachelor_Project/training_config/seq2seq_config.json' -s Output_seq2seq/ -f 

# Evaluate model

In [ ]:
!allennlp evaluate 'Output_seq2seq/model.tar.gz' '/content/gdrive/My Drive/Kaggle/LYRICS_DATASET_test.csv' --include-package information_retrieval

In [ ]:
!allennlp evaluate 'Output_seq2seq/model.tar.gz' '/content/gdrive/My Drive/Kaggle/LYRICS_DATASET_test_without_title.csv' --include-package information_retrieval

# Make Predictions

In [ ]:
!allennlp predict  'Output_seq2seq/model.tar.gz' 'predictions/rejoice_seq.json'  --predictor seq2seq

In [ ]:
import itertools

import torch
import torch.optim as optim
import allennlp
from allennlp_models.generation.dataset_readers import Seq2SeqDatasetReader
#from allennlp.data.iterators import BucketIterator
from allennlp.data.token_indexers import SingleIdTokenIndexer
from allennlp.data.tokenizers.character_tokenizer import CharacterTokenizer
from allennlp.data.vocabulary import Vocabulary
from allennlp.nn.activations import Activation
from allennlp_models.generation.models.simple_seq2seq import SimpleSeq2Seq
from allennlp.modules.attention import LinearAttention, BilinearAttention, DotProductAttention
from allennlp_models.rc.modules import StackedSelfAttentionEncoder #PytorchSeq2SeqWrapper, 
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp_models.generation.predictors import Seq2SeqPredictor
from allennlp.training.trainer import Trainer
from allennlp.training.trainer import Trainer, GradientDescentTrainer

from allennlp.data.tokenizers import (
    Token, 
    Tokenizer,
    CharacterTokenizer,
    PretrainedTransformerTokenizer,
    SpacyTokenizer,
    WhitespaceTokenizer,
)

from torch.utils.data import DataLoader
from allennlp.data import allennlp_collate
from allennlp.models import Model
import tempfile
from allennlp.training.optimizers import AdamOptimizer

In [ ]:
reader = Seq2SeqDatasetReader(
          source_tokenizer=WhitespaceTokenizer(),
          delimiter=",",
          source_token_indexers={'tokens': SingleIdTokenIndexer()},
          target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')})
train_dataset = reader.read('/content/gdrive/My Drive/Kaggle/LYRICS_DATASET_train.csv')
validation_dataset = reader.read('/content/gdrive/My Drive/Kaggle/LYRICS_DATASET_train.csv')

print("dataset", train_dataset[3])

reading instances: 217it [00:00, 1922.56it/s]
reading instances: 217it [00:00, 2181.30it/s]

dataset Instance with fields:
 	 source_tokens: TextField of length 207 with text: 
 		[@start@, across, the, sea, by, the, tennessee, skyline, they, told, me, id, find, my, hopes, and,
		my, dreams, but, i, long, to, be, in, the, bed, of, my, true, love, back, where, i, came, from,
		shes, waiting, for, me, so, ill, make, my, way, through, long, winding, country, roads, but, my,
		heart, still, beats, for, my, home, and, my, english, rose, i, told, my, dad, on, the, phone, its,
		amazing, from, the, straight, to, the, craziest, places, ive, seen, but, i, long, to, be, in, the,
		arms, of, my, true, love, like, he, loves, my, mother, he, understands, me, i, spend, my, days,
		just, traveling, and, playing, shows, but, my, heart, still, beats, for, my, home, and, my, english,
		rose, i, met, a, man, in, the, bar, down, in, memphis, he, told, me, he, went, there, to, follow,
		his, dreams, he, told, me, son, you, know, i, lost, my, true, love, for, the, same, exact, reason,
		that, you, 

In [ ]:
EN_EMBEDDING_DIM = 256
ZH_EMBEDDING_DIM = 256
HIDDEN_DIM = 256
CUDA_DEVICE = 0





def build_data_loaders(train_data: torch.utils.data.Dataset, dev_data: torch.utils.data.Dataset):
  train_loader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=allennlp_collate)
  dev_loader = DataLoader(dev_data, batch_size=8, shuffle=False, collate_fn=allennlp_collate)
  return train_loader, dev_loader



def build_trainer(model: Model, serialization_dir: str, train_loader: DataLoader, dev_loader: DataLoader) -> Trainer: 
      parameters = [
                    [n, p] for n, p in model.named_parameters() if p.requires_grad
      ]
      optimizer = AdamOptimizer(parameters)
      trainer = GradientDescentTrainer(
            model=model,
            serialization_dir=serialization_dir,
            data_loader=train_loader,
            validation_data_loader=dev_loader,
            num_epochs=1,
            optimizer=optimizer
          )
      return trainer

def run_training_loop():
    
      reader = Seq2SeqDatasetReader(
          source_tokenizer=WhitespaceTokenizer(),
          delimiter=",",
          source_token_indexers={'tokens': SingleIdTokenIndexer()},
          target_token_indexers={'tokens': SingleIdTokenIndexer(namespace='target_tokens')})
      train_dataset = reader.read('/content/gdrive/My Drive/Kaggle/LYRICS_DATASET_train.csv')
      validation_dataset = reader.read('/content/gdrive/My Drive/Kaggle/LYRICS_DATASET_train.csv')

      print("dataset", train_dataset)

      vocab = Vocabulary.from_instances(train_dataset + validation_dataset,
                                        min_count={'tokens': 3, 'target_tokens': 3})
        
        
      train_dataset.index_with(vocab)
      validation_dataset.index_with(vocab)

      train_loader, dev_loader = build_data_loaders(train_dataset, validation_dataset)

      en_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                             embedding_dim=EN_EMBEDDING_DIM)
   
      encoder = StackedSelfAttentionEncoder(input_dim=EN_EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, projection_dim=128, feedforward_hidden_dim=128, num_layers=1, num_attention_heads=8)
      source_embedder = BasicTextFieldEmbedder({"tokens": en_embedding})
      attention = DotProductAttention()
      max_decoding_steps = 20 

      model = SimpleSeq2Seq(vocab, source_embedder, encoder, max_decoding_steps,
                          target_embedding_dim=ZH_EMBEDDING_DIM,
                          target_namespace='target_tokens',
                          attention=attention,
                          beam_size=8,
                          use_bleu=True)
       
      with tempfile.TemporaryDirectory() as serialization_dir:
            trainer = build_trainer(
                model,
                serialization_dir,
                train_loader,
                dev_loader
            )
            print("Starting training")
            trainer.train()
            print("Finished training")
            print('Epoch: {}'.format(i))

            
    
      return model, reader

model, reader = run_training_loop()

validation_dataset = reader.read('/content/gdrive/My Drive/Kaggle/LYRICS_DATASET_train.csv')
predictor = Seq2SeqPredictor(model, reader)

for instance in itertools.islice(validation_dataset, 10):
    print('SOURCE:', instance.fields['source_tokens'].tokens)
    print('GOLD:', instance.fields['target_tokens'].tokens)
    print('PRED:', predictor.predict_instance(instance)['predicted_tokens'])



reading instances: 217it [00:00, 1687.67it/s]
reading instances: 217it [00:00, 557.74it/s]
building vocab: 100%|##########| 434/434 [00:00<00:00, 4726.85it/s]
You provided a validation dataset but patience was set to None, meaning that early stopping is disabled
  0%|          | 0/28 [00:00<?, ?it/s]

dataset <allennlp.data.dataset_readers.dataset_reader.AllennlpDataset object at 0x7f75104442e8>
Starting training


batch_loss: 1.8928, loss: 2.6547 ||: 100%|##########| 28/28 [00:30<00:00,  1.10s/it]
BLEU: 0.1431, batch_loss: 3.5739, loss: 2.3224 ||: 100%|##########| 28/28 [00:12<00:00,  2.21it/s]
reading instances: 217it [00:00, 2186.86it/s]


Finished training
Epoch: 0
SOURCE: [@start@, Lyric, @end@]
GOLD: [@start@, Title, @end@]
PRED: [['@@UNKNOWN@@'], ['@@UNKNOWN@@', '@@UNKNOWN@@'], [], ['@@UNKNOWN@@', '@@UNKNOWN@@', '@@UNKNOWN@@'], ['@@UNKNOWN@@', 'the'], ['@@UNKNOWN@@', 'you'], ['@@UNKNOWN@@', 'the', '@@UNKNOWN@@'], ['@@UNKNOWN@@', '@@UNKNOWN@@', '@@UNKNOWN@@', '@@UNKNOWN@@']]
SOURCE: [@start@, you, dont, have, to, say, a, thing, you, dont, have, to, share, the, pain, id, bring, and, i, dont, need, to, see, through, walls, when, the, metadata, says, it, all, postin, pictures, smiles, and, beaches, tennis, shoes, and, matching, tshirts, its, out, of, my, hands, but, its, in, my, phone, when, the, metadata, says, it, all, faith, in, timing, some, go, your, way, and, others, stay, in, place, ill, be, fighting, pain, and, the, patience, pain, and, the, patience, what, more, can, i, do, now, but, hold, on, strong, take, it, day, by, day, its, all, your, call, i, know, silence, when, its, standing, tall, and, the, metadata, s